# -------------------TP NOTE-------------------------
# Prénom & Nom : El Hadji Mamadou DIA
# Module : Code Algébrique
# Numéro Etudiant : 23010517
# ---------------------------------------------------------

# ---------------------------------------------------------------------------------------
# PARTIE 1 : Construction d’un mot de code, ajout des erreurs
# ---------------------------------------------------------------------------------------

# Question 1. Construire :

In [1]:
F = GF(19)
R = PolynomialRing(F, "X")
X = R.gen()
n = 14
k = 6
points = [F(i) for i in range(1, n + 1)]

# Question 2. Polynôme aléatoire

In [2]:
def polynome_aleatoire() :
    coeff = [F.random_element() for _ in range(k)]
    Poly = R(coeff)
    return Poly

In [3]:
P = polynome_aleatoire()
show(P)

15*X^5 + 16*X^4 + 2*X^3 + X^2 + 17*X + 12

# Question 3. Encodage

In [4]:
def encode(message) :
    encod = [message(i) for i in points]
    encod_vect = vector(encod)
    return encod_vect

In [5]:
E = encode(P)
show(E)

(6, 4, 13, 15, 0, 18, 3, 15, 7, 10, 14, 1, 16, 10)

# Question 4. Interpole

In [6]:
def interpole(mot):
    L = [(point, mot[i]) for i, point in enumerate(points)]
    poly_interpole = R.lagrange_polynomial(L) 
    return poly_interpole

In [7]:
PP = interpole(E)
show(PP)
show(PP == P)

15*X^5 + 16*X^4 + 2*X^3 + X^2 + 17*X + 12

True

# Question 5. Introduction d'erreur

In [8]:
def introduit_erreurs(mot, t):
    mot_bruite = mot.list()
    while t != 0:
        mot_bruite[randint(0, len(mot) - 1)] = F.random_element()
        t -= 1
    return vector(mot_bruite)

In [9]:
t = 4
y = introduit_erreurs(E, t)
show(y)

(4, 4, 13, 9, 0, 11, 3, 15, 7, 10, 11, 1, 16, 10)

# ----------------------------------------------------
# Partie 2. L’algorithme de décodage
# ----------------------------------------------------

# Question 6. Construction de la matrice

In [10]:
def construit_matrice(y):
    ligne_M = []
    compteur = 0
    for yi in y:
        ligne = []
        ai = points[compteur]
        ligne.extend([yi * ai**j for j in range(0, t+1)])
        ligne.extend([-ai**j for j in range(0, k + t)])
        compteur += 1
        ligne_M.append(ligne)
    M = matrix(ligne_M)
    return M

In [11]:
M = construit_matrice(y)
show("M = ",M)

'M = ' [ 4  4  4  4  4 18 18 18 18 18 18 18 18 18 18]
[ 4  8 16 13  7 18 17 15 11  3  6 12  5 10  1]
[13  1  3  9  8 18 16 10 11 14  4 12 17 13  1]
[ 9 17 11  6  5 18 15  3 12 10  2  8 13 14 18]
[ 0  0  0  0  0 18 14 13  8  2 10 12  3 15 18]
[11  9 16  1  6 18 13  2 12 15 14  8 10  3 18]
[ 3  2 14  3  2 18 12  8 18 12  8 18 12  8 18]
[15  6 10  4 13 18 11 12  1  8  7 18 11 12  1]
[ 7  6 16 11  4 18 10 14 12 13  3  8 15  2 18]
[10  5 12  6  3 18  9 14  7 13 16  8  4  2  1]
[11  7  1 11  7 18  8 12 18  8 12 18  8 12 18]
[ 1 12 11 18  7 18  7  8  1 12 11 18  7  8  1]
[16 18  6  2  7 18  6  2  7 15  5  8  9  3  1]
[10  7  3  4 18 18  5 13 11  2  9 12 16 15  1]

# Question 7. Décodage avec l’algorithme de Berlekamp-Welch

In [12]:
def berlekamp_welch(y):
    nbre_colonnes = M.ncols()
    V = VectorSpace(F, nbre_colonnes)
    base_matrice = M.right_kernel().basis_matrix()
    s = V(base_matrice[0])
    u = s[:t + 1]
    v = s[t + 1:]
    Ux = sum(u[i] * X^i for i in range(len(u)))
    Vx = sum(v[i] * X^i for i in range(len(v)))
    if Vx % Ux == 0:
        return Vx // Ux
    else:
        raise ValueError("Erreur : U(x) ne divise pas V(x).")


In [13]:
Decode = berlekamp_welch(y)
show(Decode)

15*X^5 + 16*X^4 + 2*X^3 + X^2 + 17*X + 12

# Question 8. Teste

In [14]:
def teste_Q4(nbr_test):
    Erreur = 0
    t = 4
    for _ in range(nbr_test):
        poly_alea = polynome_aleatoire()
        mot_encode = encode(poly_alea)
        mot_bruite = introduit_erreurs(mot_encode, 4)
        poly_interpole = interpole(mot_encode)
        if poly_alea == poly_interpole:
            continue
        else :
            Erreur += 1
    if Erreur == 0:
        print("L'algorithmes d'interpole fonctionne correctement sur", nbr_test, "tests.")
    else:
        print("Il y a eu", Erreur, "erreurs sur", nbr_test, "tests.")

In [15]:
teste_Q4(10)

("L'algorithmes d'interpole fonctionne correctement sur", 10, 'tests.')


In [16]:
def teste_Q7(nbr_test):
    Erreur = 0
    t = 4
    for _ in range(nbr_test):
        poly_alea = polynome_aleatoire()
        mot_encode = encode(poly_alea)
        mot_bruite = introduit_erreurs(mot_encode, 4)
        mot_decode = berlekamp_welch(mot_bruite)
        if mot_decode == mot_decode:
            continue
        else :
            Erreur += 1
    if Erreur == 0:
        print("L'algorithmes de Berlekamp Welch fonctionne correctement sur", nbr_test, "tests.")
    else:
        print("Il y a eu", Erreur, "erreurs sur", nbr_test, "tests.")

In [17]:
teste_Q7(10)

("L'algorithmes de Berlekamp Welch fonctionne correctement sur", 10, 'tests.')
